# WEO data cleaning and reshaping

This notebook loads WEO data, filters to selected indicators, cleans numeric values, pivots to a panel (Country × Year) with indicators as columns, adds a global recession flag, drops incomplete years, and writes a cleaned CSV.

In [1]:
import pandas as pd
import csv
from pathlib import Path

p = Path(r"data.csv")
if not p.exists():
    raise FileNotFoundError(p)

# detect encoding (fallback to latin-1) and sample for delimiter sniffing
encoding = "utf-8"
try:
    sample = p.read_text(encoding=encoding)[:8192]
except UnicodeDecodeError:
    encoding = "latin-1"
    sample = p.read_text(encoding=encoding)[:8192]

try:
    delim = csv.Sniffer().sniff(sample).delimiter
except Exception:
    delim = ","

df = pd.read_csv(p, sep=delim, encoding=encoding, low_memory=False, parse_dates=True)
print("shape:", df.shape)
df.head()

shape: (8626, 61)


,WEO Country Code,ISO,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,...,2022,2023,2024,2025,2026,2027,2028,2029,2030,Estimates Start After
0,512,AFG,NGDP_R,Afghanistan,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,NaN,...,"1,032.712","1,056.123",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0
1,512,AFG,NGDP_RPCH,Afghanistan,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,Units,"See notes for: Gross domestic product, consta...",NaN,...,-6.240,2.267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0
2,512,AFG,NGDP,Afghanistan,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: National Statistics Office Latest actu...,NaN,...,"1,283.442","1,350.910",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0
3,512,AFG,NGDPD,Afghanistan,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",NaN,...,14.501,17.248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0
4,512,AFG,PPPGDP,Afghanistan,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",NaN,...,86.149,91.272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0


## 1) Load CSV

Detect encoding and delimiter, then read the CSV into a DataFrame. This cell expects a local `data.csv` file.

In [2]:
Unique_Country = df['Country'].unique()

## 2) Inspect countries

Quick check to list unique country names so you can verify formatting and spot duplicates or unexpected values.

In [3]:
# count columns
print("Number of columns:", df.shape[1])

# list column names
print("Columns:", df.columns.tolist())

# counts of non-null values per column (good for spotting missing data)
print(df.count().sort_values(ascending=False))
# count columns
print("Number of columns:", df.shape[1])

# list column names
print("Columns:", df.columns.tolist())

# counts of non-null values per column (good for spotting missing data)
print(df.count().sort_values(ascending=False))

Number of columns: 61
Columns: ['WEO Country Code', 'ISO', 'WEO Subject Code', 'Country', 'Subject Descriptor', 'Subject Notes', 'Units', 'Scale', 'Country/Series-specific Notes', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', 'Estimates Start After']
WEO Country Code      8625
ISO                   8624
WEO Subject Code      8624
Country               8624
Subject Descriptor    8624
                      ... 
1984                  4104
1983                  4079
1982                  4039
1981                  3995
1980                  3861
Length: 61, dtype: int64
Number of columns: 61
Columns: ['WEO Country Code', 'ISO', 'WEO S

## 3) Column overview

Show number of columns, column names and non-null counts to locate missing-data-heavy columns that may need attention.

In [4]:
df.drop(columns=["WEO Country Code", "ISO","Country/Series-specific Notes", "Subject Notes", "Units", "Scale", "Estimates Start After", "Subject Descriptor" ], inplace=True)

codes = {"NGDPD","PCPI","TM_RPCH","TX_RPCH","LUR","LP","GGR_NGDP","GGX_NGDP","GGXWDN_NGDP","BCA_NGDPD"}
col = "WEO Subject Code"

if col not in df.columns:
    raise KeyError(f"Column {col!r} not found in dataframe")

df = df[df[col].astype(str).str.strip().isin(codes)].copy()
print("shape after filter:", df.shape)
df

shape after filter: (1960, 53)


,WEO Subject Code,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
3,NGDPD,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.278,14.501,17.248,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,PCPI,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.038,143.822,132.728,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,TM_RPCH,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-30.012,29.820,14.971,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,TX_RPCH,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.529,30.217,-2.680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,LUR,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8606,LP,Zimbabwe,7.092,7.300,7.606,7.849,8.099,8.385,8.648,8.919,...,15.904,16.229,16.624,16.993,17.360,17.729,18.104,18.488,18.882,19.285
8608,GGR_NGDP,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.348,16.599,14.649,16.581,18.549,18.698,19.117,19.197,19.277,19.277
8610,GGX_NGDP,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.573,21.476,19.968,18.641,19.053,18.257,18.231,18.201,18.143,18.110
8618,GGXWDN_NGDP,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4) Filter indicators and drop unused columns

We drop metadata columns and filter rows to the indicator codes of interest (NGDPD, PCPI, ...). Keep an eye on column names — mismatches will raise a KeyError.

In [5]:
# --- Step 1: Identify year columns ---
year_cols = df.columns[2:]  # after WEO Subject Code and Country

# --- Step 2: Clean numeric formatting ---
df[year_cols] = df[year_cols].replace({',': ''}, regex=True)
df[year_cols] = df[year_cols].apply(pd.to_numeric, errors="coerce")

# --- Step 3: Clean country names ---
df["Country"] = (
    df["Country"]
    .str.replace(" ", "_")
    .str.replace("'", "")
    .str.replace("-", "_")
)

# --- Step 4: Melt into long format ---
df_long = df.melt(id_vars=["WEO Subject Code", "Country"],
                  var_name="Year", value_name="Value")

# --- Step 5: Keep only valid year rows ---
df_long["Year"] = df_long["Year"].astype(str).str.strip()
df_long = df_long[df_long["Year"].str.fullmatch(r"\d{4}")].copy()
df_long["Year"] = df_long["Year"].astype(int)

# --- Step 6: Clean numeric values ---
df_long["Value"] = (
    df_long["Value"].astype(str)
    .str.replace(",", "")
    .replace({"": None, "nan": None})
    .astype(float)
)

# --- Step 7: Pivot using WEO Subject Code as columns ---
df_pivot = df_long.pivot_table(
    index=["Country", "Year"],
    columns="WEO Subject Code",   # use NGDPD, PCPI, etc.
    values="Value",
    aggfunc="first"
).reset_index()

# --- Step 8: Finalize ---
df_pivot.columns.name = None
df_pivot = df_pivot.set_index("Year")

df_pivot.head()


,Country,BCA_NGDPD,GGR_NGDP,GGXWDN_NGDP,GGX_NGDP,LP,LUR,NGDPD,PCPI,TM_RPCH,TX_RPCH
Year,,,,,,,,,,,
2002,Afghanistan,33.908,6.845,NaN,6.943,21.000,NaN,4.367,31.263,NaN,NaN
2003,Afghanistan,29.616,9.826,NaN,11.927,22.645,NaN,4.553,42.413,36.222,49.541
2004,Afghanistan,37.216,12.676,NaN,15.069,23.554,NaN,5.146,49.351,-0.427,-8.436
2005,Afghanistan,30.226,14.733,NaN,15.651,24.411,NaN,6.167,54.566,55.010,41.968
2006,Afghanistan,20.844,18.946,NaN,18.262,25.443,NaN,6.925,58.269,-2.198,-6.919


## 5) Clean numeric values, standardize country names, and reshape

Steps:
- Identify year columns and coerce them to numeric (remove commas).
- Standardize country strings to avoid duplication (underscores, remove quotes/hyphens).
- Melt to long format (WEO Subject Code, Country, Year, Value).
- Keep only rows where Year looks like YYYY and ensure Value is numeric.
- Pivot so each row is Country × Year and columns are indicators.

If duplicate (Country, Year, Indicator) rows exist, the code uses `aggfunc='first'`. Inspect duplicates if results look wrong.

In [6]:
# Convert Year index to integer
df_pivot.index = df_pivot.index.astype(int)

# Define global recession years
global_recession_years = [
    1981, 1982, 1983,
    1990, 1991, 1992, 1993,
    2008, 2009, 2010,
    2020, 2021
]

# Add Global_Recession column
df_pivot["Global_Recession"] = df_pivot.index.isin(global_recession_years).astype(int)

df_pivot


,Country,BCA_NGDPD,GGR_NGDP,GGXWDN_NGDP,GGX_NGDP,LP,LUR,NGDPD,PCPI,TM_RPCH,TX_RPCH,Global_Recession
Year,,,,,,,,,,,,
2002,Afghanistan,33.908,6.845,NaN,6.943,21.000,NaN,4.367,31.263,NaN,NaN,0
2003,Afghanistan,29.616,9.826,NaN,11.927,22.645,NaN,4.553,42.413,36.222,49.541,0
2004,Afghanistan,37.216,12.676,NaN,15.069,23.554,NaN,5.146,49.351,-0.427,-8.436,0
2005,Afghanistan,30.226,14.733,NaN,15.651,24.411,NaN,6.167,54.566,55.010,41.968,0
2006,Afghanistan,20.844,18.946,NaN,18.262,25.443,NaN,6.925,58.269,-2.198,-6.919,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2026,Zimbabwe,3.063,18.698,NaN,18.257,17.729,NaN,39.491,1204688.561,NaN,NaN,0
2027,Zimbabwe,2.927,19.117,NaN,18.231,18.104,NaN,40.925,1300690.916,NaN,NaN,0
2028,Zimbabwe,2.895,19.197,NaN,18.201,18.488,NaN,42.340,1404343.758,NaN,NaN,0


## 6) Add global recession flag

Convert the Year index to integer and add a `Global_Recession` column marking selected years. Adjust the list of recession years if your definition differs.

In [8]:
# Remove rows with any NaN values
df_pivot = df_pivot.dropna()
df_pivot

,Country,BCA_NGDPD,GGR_NGDP,GGXWDN_NGDP,GGX_NGDP,LP,LUR,NGDPD,PCPI,TM_RPCH,TX_RPCH,Global_Recession
Year,,,,,,,,,,,,
2011,Albania,-12.765,25.108,57.474,28.583,2.905,14.0,13.052,92.892,3.639,1.013,0
2012,Albania,-10.082,24.497,59.865,27.897,2.900,13.4,12.470,94.783,-7.604,-0.407,0
2013,Albania,-9.144,23.696,67.444,28.849,2.895,15.9,12.936,96.608,-0.612,8.288,0
2014,Albania,-10.670,25.963,64.766,31.356,2.889,17.5,13.404,98.167,5.418,3.199,0
2015,Albania,-8.508,26.129,67.993,30.396,2.881,17.1,11.525,100.000,0.070,5.908,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2026,Uruguay,-1.662,28.217,59.226,30.919,3.520,8.0,85.164,120.333,5.311,3.505,0
2027,Uruguay,-1.826,28.169,59.041,30.753,3.528,8.0,90.702,126.028,5.100,3.432,0
2028,Uruguay,-1.875,28.160,58.910,30.572,3.535,8.0,96.223,131.862,5.022,3.371,0


## 7) Remove incomplete rows

Dropping all rows with any NaN removes years/countries that lack any indicator value. Consider imputing or relaxing this if you need more coverage.

In [12]:
df_pivot["Country"].unique()

array(['Albania', 'Algeria', 'Australia', 'Austria', 'Barbados',
       'Belgium', 'Bolivia', 'Bosnia_and_Herzegovina', 'Brazil',
       'Bulgaria', 'Cabo_Verde', 'Canada', 'Chile', 'Colombia',
       'Costa_Rica', 'Croatia', 'Cyprus', 'Czech_Republic', 'Denmark',
       'Dominican_Republic', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Hungary', 'Iceland', 'Indonesia', 'Ireland',
       'Islamic_Republic_of_Iran', 'Israel', 'Italy', 'Japan', 'Jordan',
       'Kazakhstan', 'Korea', 'Latvia', 'Lebanon', 'Lithuania',
       'Luxembourg', 'Malta', 'Mexico', 'Morocco', 'Netherlands',
       'New_Zealand', 'Nigeria', 'North_Macedonia', 'Norway', 'Pakistan',
       'Panama', 'Paraguay', 'Peru', 'Poland', 'Portugal', 'Romania',
       'Saudi_Arabia', 'Serbia', 'Seychelles', 'Slovak_Republic',
       'Slovenia', 'South_Africa', 'Spain', 'Sweden', 'Switzerland',
       'Syria', 'Taiwan_Province_of_China', 'Trinidad_and_Tobago',
       'Türkiye', 'United_Kingdom', 'United_States',

## 8) Export

Write the cleaned panel to CSV for downstream analysis or modeling.

In [10]:
# Write dataframe to CSV
df_pivot.to_csv("weo_data_clean.csv", index=True)
